In [ ]:
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from datetime import timedelta

from tvb_inversion.pymc3.plot import plot_posterior_samples
from tvb_inversion.base.diagnostics import (zscore, shrinkage)
from tvb_inversion.pymc3.plot import (
    plot_posterior_samples_model_parameters,
    plot_posterior_samples_global_parameters,
    posterior_pairplot
)

%load_ext autoreload
%autoreload 2

In [ ]:
# run_id = "2023-02-01_1200"  # around ground truth
# run_id = "2023-02-02_1233"  # around ground truth
# run_id = "2023-02-03_1126"  # shifted from ground truth
run_id = "2023-02-04_0937"  # shifted from ground truth
# run_id = "2023-02-10_1605"  # around ground truth
# run_id = "2023-02-24_1844"  # around ground truth

inference_data = az.from_netcdf(f"pymc3_data/{run_id}_idata.nc")
inference_summary = pd.read_json(f"pymc3_data/{run_id}_isummary.json")
with open(f"pymc3_data/{run_id}_iparams.json", "r") as f:
    inference_params = json.load(f)
    f.close()
with open(f"pymc3_data/{run_id}_sim_params.json", "r") as f:
    sim_params = json.load(f)
    f.close()

# inference_data = az.from_netcdf(f"pymc3_data/{run_id}.nc")
# inference_summary = pd.read_json(f"pymc3_data/{run_id}.json")

In [ ]:
inference_params

In [ ]:
X = inference_data.observed_data.x_obs.values

In [ ]:
chains = len(inference_data.sample_stats.chain)
draws = len(inference_data.sample_stats.draw)

In [ ]:
print("#chains: ", chains)
print("#draws: ", draws)
divergent = inference_data.sample_stats["diverging"].values
print("Number of Divergent: %d" % divergent.nonzero()[0].size)
divperc = (divergent.nonzero()[0].size / (chains * draws)) * 100
print("Percentage of Divergent: %.1f" % divperc)
print("Mean tree accept: %.2f" % inference_data.sample_stats.acceptance_rate.values.mean())
print("Mean tree depth: %.2f" % inference_data.sample_stats.tree_depth.values.mean())
print("Sampling time:", str(timedelta(seconds=inference_data.sample_stats.sampling_time)))

In [ ]:
inference_summary.loc[
    [f"model_a_star[{i}]" for i in range(len(inference_params["model_a"]))] + ["coupling_a_star", "nsig_star",
                                                                               "amplitude_star", "offset_star", "measurement_noise_star"]
    ]

In [ ]:
ground_truth = np.array(
    [v for v in sim_params["model_a"]] + [sim_params["coupling_a"], sim_params["nsig"], 1.0, 0.0, 0.0])
inference_summary_ = inference_summary.loc[
    [f"model_a[{i}]" for i in range(len(inference_params["model_a"]))] + ["coupling_a", "nsig",
                                                                          "amplitude", "offset", "measurement_noise"]
    ]
inference_summary_.insert(0, "ground_truth", ground_truth)
inference_summary_

In [ ]:
df = pd.DataFrame(
    data=inference_data.posterior.model_a.values.reshape(-1, inference_data.posterior.model_a.values.shape[-1]),
    columns=[f"Region {i+1}" for i in range(len(inference_params["model_a"]))]
)

plot_posterior_samples_model_parameters(df, sim_params["model_a"])

In [ ]:
from scipy.stats import spearmanr

posterior_model_a = inference_summary.loc[[f"model_a[{i}]" for i in range(len(inference_params["model_a"]))]][["mean"]].to_numpy()
gt_model_a = np.array([v for v in sim_params["model_a"]])

spearman_correlation = spearmanr(gt_model_a, posterior_model_a)

print(spearman_correlation.correlation, spearman_correlation.pvalue)

In [ ]:
fig_spearman = plt.figure(figsize=(8, 5))
plt.plot(posterior_model_a, gt_model_a, linewidth=0, marker="*", markersize=12, color="blue")
plt.tick_params(axis="both", labelsize=16)

In [ ]:
data = {
    f"model_a_R{i+1}": inference_data.posterior.model_a.values.reshape(-1, inference_data.posterior.model_a.values.shape[-1])[:, i] for i in range(len(inference_params["model_a"]))
}
data["coupling_a"] = inference_data.posterior.coupling_a.values.flatten()
df = pd.DataFrame(data)

posterior_pairplot(df, size=5)

In [ ]:
data={
    "coupling_a": inference_data.posterior.coupling_a.values.flatten(),
    "nsig": inference_data.posterior.nsig.values.flatten(),
    "measurement_noise": inference_data.posterior.measurement_noise.values.flatten(),
}
df = pd.DataFrame(data)

plot_posterior_samples_global_parameters(df, sim_params)

In [ ]:
posterior_x_obs = inference_data.posterior_predictive.x_obs.values.reshape(
    (chains*draws, *X.shape))

In [ ]:
num_regions = 6
f4, axes4 = plt.subplots(nrows=num_regions, ncols=1, figsize=(25, num_regions*12))
for i in range(num_regions):
    ax = axes4[i]

    ax.plot(np.percentile(posterior_x_obs[:, :, 0, i], [2.5, 97.5], axis=0).T,
            "k", label=r"$V_{95\% PP}(t)$")
    ax.plot(X[:, 0, i], label="V_observed")
    ax.set_title(f"Region {i+1}", fontsize=16)
    ax.legend(fontsize=16)
    ax.set_xlabel("time (ms)", fontsize=16)
    ax.tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
posterior_dWt_star = inference_data.posterior.dWt_star.values[:, :, :, 0, :].reshape(
    (chains*draws, *X.shape))

In [ ]:
inference_summary.loc[
    inference_summary.index.str.contains("dWt_star")].mean()

In [ ]:
num_regions = 6
f10, axes10 = plt.subplots(nrows=num_regions, ncols=1, figsize=(25, num_regions*12))
for i in range(num_regions):
    ax = axes10[i]

    ax.plot(np.percentile(posterior_dWt_star[:, :, 0, i], [2.5, 97.5], axis=0).T,
            "k", label=r"$V_{95\% PP}(t)$")
    #ax.plot(X[:, 0, i], label="V_observed")
    ax.set_title(f"Region {i+1}", fontsize=16)
    ax.legend(fontsize=16)
    ax.set_xlabel("time (ms)", fontsize=16)
    ax.tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
waic = az.waic(inference_data)
loo = az.loo(inference_data)
print("WAIC: ", waic.waic)
print("LOO: ", loo.loo)

In [ ]:
def get_posterior_mean(idata, params):
    posterior = np.concatenate(
        [idata.posterior[param].values.reshape(-1, idata.posterior[param].values.shape[-1]) if idata.posterior[param].values.ndim == 3
         else idata.posterior[param].values.flatten()[..., np.newaxis] for param in params],
        axis=1
    )
    # posterior = np.asarray([idata.posterior[param].values.reshape((idata.posterior[param].values.size,)) for param in params])
    return posterior.mean(axis=0)

def get_posterior_std(idata, params):
    posterior = np.concatenate(
        [idata.posterior[param].values.reshape(-1, idata.posterior[param].values.shape[-1]) if idata.posterior[param].values.ndim == 3
         else idata.posterior[param].values.flatten()[..., np.newaxis] for param in params],
        axis=1
    )
    # posterior = np.asarray([idata.posterior[param].values.reshape((idata.posterior[param].values.size,)) for param in params])
    return posterior.std(axis=0)

In [ ]:
# zscores
posterior_mean = get_posterior_mean(inference_data,
                                    ["model_a", "coupling_a", "nsig"])

posterior_std = get_posterior_std(inference_data,
                                  ["model_a", "coupling_a", "nsig"])

ground_truth = np.array(
    [v for v in sim_params["model_a"]] + [sim_params["coupling_a"], sim_params["nsig"]])

zscores = zscore(ground_truth, posterior_mean, posterior_std)

In [ ]:
# shrinkages
posterior_std = get_posterior_std(inference_data,
    ["model_a_star", "coupling_a_star", "nsig_star"])

prior_std = np.ones((12,))

shrinkages = shrinkage(prior_std, posterior_std)

In [ ]:
def zscore_gt(true_mean, post_mean):
    return np.abs((post_mean - true_mean) / true_mean)

def zscore_prior(true_mean, post_mean, prior_std):
    return np.abs((post_mean - true_mean) / prior_std)

In [ ]:
prior_std = np.hstack(
    (np.array([0.75 for _ in range(len(inference_params["model_a"]))]), (0.5*ground_truth)[len(inference_params["model_a"]):]))

zscores_gt = zscore_gt(ground_truth, posterior_mean)
zscores_prior = zscore_prior(ground_truth, posterior_mean, prior_std)

In [ ]:
f5 = plt.figure(figsize=(12,8))
for i in range(len(sim_params["model_a"])):
    plt.plot(shrinkages[i], zscores[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[-2], zscores[-2], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[-1], zscores[-1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage", size=16)
plt.ylabel("posterior zscore", size=16)
plt.xlim(xmax=1.05)
plt.legend(fontsize=16)
plt.tick_params(axis="both", labelsize=16)
plt.plot();

In [ ]:
f6 = plt.figure(figsize=(12,8))
for i in range(len(sim_params["model_a"])):
    plt.plot(shrinkages[i], zscores_gt[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[-2], zscores_gt[-2], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[-1], zscores[-1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage", size=16)
plt.ylabel("posterior zscore", size=16)
plt.xlim(xmax=1.05)
plt.legend(fontsize=16)
plt.tick_params(axis="both", labelsize=16)
plt.plot();

In [ ]:
f7 = plt.figure(figsize=(12,8))
for i in range(len(sim_params["model_a"])):
    plt.plot(shrinkages[i], zscores_prior[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[-2], zscores_prior[-2], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[-1], zscores[-1], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.xlabel("posterior shrinkage", size=16)
plt.ylabel("posterior zscore", size=16)
plt.xlim(xmax=1.05)
plt.legend(fontsize=16)
plt.tick_params(axis="both", labelsize=16)
plt.plot();